# 认识python并行

先了解些基本概念。

什么是 concurrency？什么是parallelism？python并行方法中threading，asyncio，和multiprocessing有啥区别？什么时候用concurrency，有哪些module？

后面不会特别提到线程和进程的概念，但是实际上thread就是线程管理并行的Python模块，multiprocessing就是进程相关的并行模块。在大多数操作系统中，每个程序在一个进程中运行。通常，我们通过双击一个应用程序的图标来启动程序。进程有自己的地址空间，数据栈和其他的辅助数据来追踪执行过程；系统会管理所有进程的执行，通过调度程序来分配计算资源等。一个进程中可以有多个线程。

然后主要根据 [Parallel Python: Analyzing Large Datasets](https://github.com/pydata/parallel-tutorial) 掌握一些具体的代码，基本都直接使用原教程的文件了，在 parallel-tutorial-master 文件夹下。

开始之前先准备下数据。

在jupyter下打开console，运行7-parallel-programming/parallel-tutorial-master 文件夹下的 prep.py。

```Shell
cd 7-parallel-programming/parallel-tutorial-master
python prep.py
```

## concurrency

concurrency 是多义词。在python中，同时发生的事情被叫做不同的名字：thread，task，process，不过他们都是指一系列按顺序执行的指令。

可以把并行当作大脑中不同的思维线，思考时候，可以在某一条线上暂停，并切换到另一条线上，每条线的状态都被记下来，以便后面可以返回找到中断点。

那么为什么python要用这些不同的词表示相同的概念呢？因为threads，tasks，processes这些词本质上做的事一样，不过具体的细节是有所不同的，它们代表了不同的细节表达。后面会慢慢补充。

当谈论稍微深入一点时，要小心，只有multiprocessing 实际上在同一时间运行多个trains。Threading和 asyncio 都是在单一处理器上运行的，一次只能run一个，它们只是聪明地找到了加速过程的方法，所以即便它们不是真的并发，也称他们为concurrency。

threading和asyncio中 threads 或 tasks 执行的顺序是大不同的，在threading中，操作系统知道每个thread并能在任意时间中断它来开始一个不同的thread，这杯称为 pre-emptive multitasking 抢先多任务处理，因为操作系统能预先处理线程。

pre-emptive multitasking 很方便，thread 中的代码不需要做任何事情就能完成转换。不过因为其“at any time”的特点。它也可以是难的。其switch 可以发生在一个单一的python语句中间，即使是一个trivial one，比如 x=x+1

Asyncio 使用了 cooperative multitasking 协调式多任务。tasks 必须通过声明它们什么时候准备好了被换出才能协作。这意味着task的代码必须有所动作才能使switch发生。

多出来的这一步的好处是你总是指导task什么时候被换出了。它不会在一个python语句中被换出，除非这个语句是被标记的。

## Parallelism

前面说的是单个处理器上的concurrency。当涉及同时使用多核时，就是multiprocessing。使用multiprocessing，python创建心得processes。一个process可以被任务是一个完全不同的程序。可以认为每个process在自己的python解释器上运行。

因为它们是不同的processes，每个程序的训练可以在不同的核上运行。这是真正同一时间多个程序运行。

总结下concurrency和parallelism：

|Concurrency Type|Switching Decision|Number of Processors|
|-|-|-|
|Pre-emptive multitasking (threading)|The operating system decides when to switch tasks external to Python.|1|
|Cooperative multitasking (asyncio)|The tasks decide when to give up control.|1|
|Multiprocessing (multiprocessing)|The processes all run at the same time on different processors.|Many|

## 什么时候concurrency有用？

有两类问题：CPU-bound和 I/O bound。

I/O bound 问题使得程序变慢，即内存/网络比CPU慢很多的时候。

另一类是CPU-bound，是CPU限制了速度。

并行程序总是要花费写功夫的，所以要确定提速是否值得花费功夫。

|I/O-Bound Process	|CPU-Bound Process|
|-|-|
|Your program spends most of its time talking to a slow device, like a network connection, a hard drive, or a printer.	|You program spends most of its time doing CPU operations.|
|Speeding it up involves overlapping the times spent waiting for these devices.	|Speeding it up involves finding ways to do more computations in the same amount of time.|

先看如何加速 I/O-Bound 程序。一个例子：从网页下载文件，网络拥堵。

先看一个无并行的版本。

In [1]:
import requests
import time


def download_site(url, session):
    with session.get(url) as response:
        print(f"Read {len(response.content)} from {url}")


def download_all_sites(sites):
    with requests.Session() as session:
        for url in sites:
            download_site(url, session)


sites = ["https://www.jython.org","http://olympus.realpython.org/dice",] * 8
start_time = time.time()
download_all_sites(sites)
duration = time.time() - start_time
print(f"Downloaded {len(sites)} in {duration} seconds")

Read 10394 from https://www.jython.org
Read 275 from http://olympus.realpython.org/dice
Read 10394 from https://www.jython.org
Read 275 from http://olympus.realpython.org/dice
Read 10394 from https://www.jython.org
Read 275 from http://olympus.realpython.org/dice
Read 10394 from https://www.jython.org
Read 275 from http://olympus.realpython.org/dice
Read 10394 from https://www.jython.org
Read 275 from http://olympus.realpython.org/dice
Read 10394 from https://www.jython.org
Read 275 from http://olympus.realpython.org/dice
Read 10394 from https://www.jython.org
Read 275 from http://olympus.realpython.org/dice
Read 10394 from https://www.jython.org
Read 275 from http://olympus.realpython.org/dice
Downloaded 16 in 5.11754035949707 seconds


最后打印出了一步步下载所花费的总时间。同步程序最大的问题就是慢了，因为是从网上下载的。所以受限于IO速度。现在先试试threading。

In [2]:
import concurrent.futures
import requests
import threading
import time


thread_local = threading.local()


def get_session():
    if not hasattr(thread_local, "session"):
        thread_local.session = requests.Session()
    return thread_local.session


def download_site(url):
    session = get_session()
    with session.get(url) as response:
        print(f"Read {len(response.content)} from {url}")


def download_all_sites(sites):
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        executor.map(download_site, sites)


sites = ["https://www.jython.org", "http://olympus.realpython.org/dice",] * 8
start_time = time.time()
download_all_sites(sites)
duration = time.time() - start_time
print(f"Downloaded {len(sites)} in {duration} seconds")

Read 10394 from https://www.jython.orgRead 10394 from https://www.jython.org

Read 10394 from https://www.jython.org
Read 10394 from https://www.jython.org
Read 10394 from https://www.jython.org
Read 275 from http://olympus.realpython.org/dice
Read 275 from http://olympus.realpython.org/dice
Read 275 from http://olympus.realpython.org/dice
Read 10394 from https://www.jython.org
Read 275 from http://olympus.realpython.org/dice
Read 275 from http://olympus.realpython.org/dice
Read 10394 from https://www.jython.org
Read 275 from http://olympus.realpython.org/dice
Read 275 from http://olympus.realpython.org/dice
Read 10394 from https://www.jython.org
Read 275 from http://olympus.realpython.org/dice
Downloaded 16 in 1.5960195064544678 seconds


可以看到，在只执行几次的情况下，有threading的就快了很多。并且修改的代码并不是很多，download_all_sites() 从 calling the function once per site 变味了 a more complex structure.  首先创建了一个 ThreadPoolExecutor，它就是 Thread + Pool + Executor。

Thread就是之前提到的，Pool 会创建一个线程池，这些线程可以同时运行。Executor 是控制线程池中每个线程how and when 运行的。执行pool中的request。

创建 ThreadPoolExecutor 实例作为一个context manager，然后就可以使用map函数（mapreduce的map意思）。每个thread需要创建自己的requests.Session()对象，而其不是线程安全的，即共享变量处理是不被保障的。

有线程安全的数据结构，比如Queue，其会使用 threading.Lock 来保证只有一个线程可以同时访问代码块或内存位。另一个策略是thread local storage。Threading.local()创建一个类似全局变量的对象，其只属于一个thread。

那么为什么会快呢？因为thread之间不必互相等待：

![](pictures/QQ截图20201024095650.png)

线程之间的交互方式可能很微妙，很难察觉。这些交互可能会导致race conditions竞态条件，这些race conditions 经常会导致很难找到的随机的、间歇的bug。

竞态条件是一整个类的微妙bug，它们可以而且经常发生在多线程代码中。竞态条件的发生是因为程序员没有充分保护数据访问以防止线程相互干扰。在编写线程代码时，需要采取额外的步骤来确保线程安全。

这里所发生的是操作系统在控制你的线程何时运行以及它何时被换出以让另一个线程运行。这种线程交换可以在任何时候发生，即使在执行Python语句的子步骤时也是如此。作为一个快速示例，看一下这个函数

In [3]:
import concurrent.futures


counter = 0


def increment_counter(fake_value):
    global counter
    for _ in range(10):
        counter += 1


fake_data = [x for x in range(50)]
counter = 0
with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
    executor.map(increment_counter, fake_data)

主要是有一个global变量counter，其不被以任何形式保护，所以是线程不安全的。

下面看看 asyncio的版本。asyncio的概念主要是一个称为 event loop的python对象，其控制着task的运行。event loop很清楚每个task以及它们的状态。一个简化的event loop有两个状态，一是ready state，表示任务有工作要做并且已经准备好运行，一是waiting state，即等待状态，意味着任务在等待某些外部事物完成。

可以简单认为uevent loop 包含两类tasks，每类对应每种状态。选择ready tasks中的一个然后开始运行，知道task将控制权交回给event loop。

当 running task 给回控制权到event loop，event loop会将task放到 ready或waiting的list，然后再看waiting list中的每个任务，判断是否ready。

当所有任务再次被排序到正确的列表中，事件循环将选择下一个要运行的任务，并重复此过程。经过简化的事件循环选择等待时间最长的任务并运行它。此过程重复进行，直到事件循环完成。

asyncio的一个重要特点是任务不会在没有意图的情况下放弃控制。他们在运行过程中从不被打断。这允许我们在异步中比在线程中更容易地共享资源。不必担心如何使代码线程安全。

python中有两个关键字：async 和 await.

根据上面的讨论，可以将await视为一种魔法，它允许任务将控制权交还给事件循环。当代码等待函数调用时，这是一个信号，表明调用可能是需要一段时间的事情，并且任务应该放弃控制。

最简单的做法是将async看作是Python的一个标志，告诉它将要定义的函数使用await。在某些情况下，这并不是完全正确的，比如异步生成器，但在许多情况下，它都适用，并且提供了一个简单的模型。

总之，需要记住任何调用 await的函数 需要被标记为 async。

asynico的版本

![](pictures/QQ截图20201024102647.png)

异步的问题之一是如果其中一个任务不合作，那么多任务合作的所有优势就会被抛弃。

下面看看multiprocessing。其利用了多核CPU的优势。

在jupyter中，不能直接方便地使用multiprocessing，必须要把pool.map中使用的函数单独放到一个文件中

In [5]:
import multiprocessing
import time
import multiprocess_def

def find_sums(numbers):
    with multiprocessing.Pool() as pool:
        pool.map(multiprocess_def.cpu_bound, numbers)

numbers = [5_000_000 + x for x in range(20)]

start_time = time.time()
find_sums(numbers)
duration = time.time() - start_time
print(f"Duration {duration} seconds")

Duration 4.423195123672485 seconds


到此之前的代码都是在单一CPU或核上运行的。multiprocessing 是一个标准库来帮助把CPU工作分散到多个CPU上。因为它是通过在每个CPU上创建新的python解释器实例来运行，所以其并不会像在当前python解释器上开一个线程那么快，所以它是个重量级的运算，并不是总好用的。

使用并行要慎重，过早的去优化代码并不是好事。要先判断自己的程序是CPU-bound还是I/O-bound。前者需要multiprocessing，thread和asyncio不能起帮助，后者可以用thread或asyncio来解决，使用哪一个可以用这句话来帮助决定：“Use asyncio when you can, threading when you must.”

因为我更多时候需要的是计算方面的并行，所以muliprocessing用的可能更多。

所以接下来稍微看看 并行计算相关的内容。

首先了解下 Python Global Interpreter Lock（GIL）。

## Python Global Interpreter Lock（GIL）

Python Global Interpreter Lock 或 GIL 是一个只允许一个thread控制python解释器的锁。

这意味着在任何时间点只能有一个线程处于执行状态。对于执行单线程程序的开发人员而言，GIL的影响并不明显，但它可能是CPU绑定和多线程代码的性能瓶颈。

由于即使在具有多个CPU内核的多线程体系结构中，GIL一次只允许执行一个线程，因此GIL在Python圈中可谓的“臭名昭著”。这里将了解GIL如何影响Python程序的性能，以及如何减轻GIL对代码的影响。

Python使用reference counting引用计数进行内存管理。这意味着用Python创建的对象有一个引用计数变量，该变量跟踪指向该对象的引用数。当此计数达到零时，将释放对象占用的内存。

In [6]:
import sys
a = []
b = a
sys.getrefcount(a)

3

空列表对象[]的引用计数为3。列表对象被a，b引用并且传递给了sys.getrefcount()。

问题是该引用计数变量需要保护，以防止两个线程同时增加或减少其值时发生的竞争。如果发生这种情况，则可能导致从未释放的内存泄漏，或者更糟糕的是，在仍然存在对该对象的引用的情况下，错误地释放了内存。这可能会导致崩溃或Python程序中的其他“怪异”错误。

通过将锁添加到跨线程共享的所有数据结构中，以便不会被不一致地修改，可以保持此引用计数变量的安全。

但是，将锁添加到每个对象或对象组意味着存在多个锁，这可能会引起另一个问题-死锁（死锁只有在有多个锁的情况下才会发生）。另一个副作用是由于重复获取和释放锁而导致性能下降。

GIL是解释器本身的单个锁，它添加了一个规则，即任何Python字节码的执行都需要获取解释器锁。这样可以防止死锁（因为只有一个锁）并且不会带来太多的性能开销。但这实际上使所有受CPU约束的Python程序都是单线程的。

尽管GIL也用于其他语言（例如Ruby），但其并不是解决此问题的唯一方法。某些语言通过使用引用计数以外的方法（例如垃圾回收）来避免对线程安全的内存管理使用GIL的要求。

另一方面，这意味着这些语言通常必须通过添加其他性能提升功能（如JIT编译器）来弥补GIL的单线程性能优势的损失。

那么，为什么在Python中使用了一种似乎如此阻碍的方法？Python开发人员是否会做出错误的决定？

用Larry Hastings的话来说， GIL的设计决定是使Python像今天一样流行的原因之一。

从操作系统没有线程概念时候，Python就已经存在了。Python被设计为易于使用，以加快开发速度，越来越多的开发人员开始使用它。

现有的C库正在编写许多扩展，这些C需要在Python中提供其功能。为了防止不一致的更改，这些C扩展需要GIL提供的线程安全内存管理。

GIL易于实现，并且很容易添加到Python中。由于只需要管理一个锁，因此它可以提高单线程程序的性能。

非线程安全的C库变得易于集成。这些C扩展成为Python被不同社区轻易采用的原因之一。

如您所见，GIL是CPython开发人员在Python生命早期面临的一个难题的务实解决方案。

当查看典型的Python程序（或与此相关的任何计算机程序）时，受CPU限制的性能与受I / O限制的性能之间是有区别的。

受CPU约束的程序是将CPU推到极限的程序。这包括进行数学计算的程序，例如矩阵乘法，搜索，图像处理等。

受I / O约束的程序是花费时间等待输入/输出的程序，这些程序可能来自用户，文件，数据库，网络等。受I / O约束的程序有时必须等待大量时间，直到它们进入由于源可能需要在输入/输出准备好之前进行自己的处理，因此可以从源那里获得他们需要的东西，例如，用户考虑要在输入提示中输入什么内容或在其输入中运行数据库查询的过程。

让我们看一个执行倒计时的简单的受CPU约束的程序

In [7]:
# single_threaded.py
import time
from threading import Thread

COUNT = 50000000

def countdown(n):
    while n>0:
        n -= 1

start = time.time()
countdown(COUNT)
end = time.time()

print('Time taken in seconds -', end - start)

Time taken in seconds - 4.0197718143463135


现在将其修改为并行的。

In [8]:
# multi_threaded.py
import time
from threading import Thread

COUNT = 50000000

def countdown(n):
    while n>0:
        n -= 1

t1 = Thread(target=countdown, args=(COUNT//2,))
t2 = Thread(target=countdown, args=(COUNT//2,))

start = time.time()
t1.start()
t2.start()
t1.join()
t2.join()
end = time.time()

print('Time taken in seconds -', end - start)

Time taken in seconds - 4.008107662200928


两个版本花费的时间几乎相同。在多线程版本中，GIL阻止了CPU-bound的线程并行执行。

由于线程在等待I / O时共享锁，因此GIL对受I / O-bound 的多线程程序的性能影响不大。

但是如上例所示，线程完全受CPU-bound的程序（例如使用线程处理映像的程序）不仅会由于锁定而变为单线程，而且与将其编写为完全单线程的方案相比，执行时间也会增加。

这种增加是**锁增加了获取和释放开销的结果**。

那么为什么还没有删除GIL？

Python的开发人员对此有很多抱怨，但是像Python这样流行的语言在不引起向后不兼容的问题的情况下，不能带来与删除GIL一样大的变化。

GIL显然可以删除，并且开发人员和研究人员过去已经做过多次，但是所有这些尝试都破坏了现有的C扩展，这在很大程度上取决于GIL提供的解决方案。

当然，还有其他解决方案可以解决GIL所解决的问题，但是其中一些解决方案会降低单线程和多线程I / O绑定程序的性能，而其中某些解决方案太困难了。毕竟，您不希望新版本发布后现有的Python程序运行速度变慢，对吧？

Python的创建者Guido van Rossum在2007年9月的文章“删除GIL并不容易”中向社区做出了回答：

“只有在单线程程序（以及多线程但受I / O-bound的程序）的性能不降低的情况下，我才欢迎在Py3k中安装一组补丁程序”

此后的任何尝试都未满足此条件。

为什么在Python 3中未将其删除？

Python 3确实有机会从头开始启动许多功能，并且在此过程中破坏了一些现有的C扩展，这些扩展随后需要进行更新并移植到Python 3才能使用。这就是早期版本 Python 3的社区采用速度较慢的原因。

但是为什么不将GIL一起删除呢？

与单线程性能相比，删除GIL会使Python 3的速度比Python 2慢，并且您可以想象会导致什么。您无法与GIL的单线程性能优势抗衡。因此，结果是Python 3仍然具有GIL。

如何处理Python的GIL呢？可以尝试以下几种方法。

Multi-processing vs multi-threading：最流行的方法是使用multi-processing，使用多个进程而不是线程。每个Python进程都有自己的Python解释器和内存空间，因此GIL不会成为问题。Python有一个multiprocessing模块，可以让我们轻松地创建如下过程：

In [9]:
from multiprocessing import Pool
import time
import multiprocess_def

COUNT = 50000000

pool = Pool(processes=2)
start = time.time()
r1 = pool.apply_async(multiprocess_def.countdown, [COUNT//2])
r2 = pool.apply_async(multiprocess_def.countdown, [COUNT//2])
pool.close()
pool.join()
end = time.time()
print('Time taken in seconds -', end - start)

Time taken in seconds - 2.287207841873169


与多线程版本相比，性能提高了。时间并没有减少到我们上面看到的一半，因为流程管理有其自己的开销。多个进程比多个线程重，而这可能会成为扩展瓶颈。

备选的Python解释器： Python具有多种解释器实现。最受欢迎的分别是用C，Java，C＃和Python编写的CPython，Jython，IronPython和PyPy。GIL仅存在于原始Python实现中，即CPython。如果您的程序及其库可用于其他实现之一，则也可以尝试一下。

稍等一下：尽管许多Python用户都利用了GIL的单线程性能优势。多线程程序员不必烦恼，因为Python社区中一些最聪明的人正在努力从CPython中删除GIL。一种这样的尝试被称为“ Gilectomy”。

Python GIL通常被认为是一个神秘而困难的话题。但是请记住，作为Pythonista，通常只有在编写C扩展或在程序中使用CPU绑定多线程时才受到它的影响。

总之，在了解更复杂的分布式编程之前，比较不错的一个加速自己程序的方式是使用multiprocessing。